# ダイワメジャー産駒の分析

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../tmp/result.csv")

/var/folders/tn/j8hdvbw52z938h6gxwsg4b4w0000gn/T/ipykernel_46591/891290719.py:1: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../tmp/result.csv")


In [3]:
# 父ダイワメジャー or 母父ダイワメジャー

sire = (df["父馬名"] == "")
ground_type = (df["芝・ダ"] == "芝")
cond = sire & ground_type
df_daiwa = df[cond]

In [4]:
df_daiwa.sample(1)

ValueError: a must be greater than 0 unless no samples are taken

In [48]:
# 距離を指定された範囲で分類
def classify_distance(distance):
    if 1000 <= distance <= 1399:
        return '1000-1399m'
    elif 1400 <= distance <= 1799:
        return '1400-1799m'
    elif 1800 <= distance <= 2199:
        return '1800-2199m'
    elif 2200 <= distance <= 2600:
        return '2200-2600m'
    elif 2600 < distance:
        return '2601m-'
    elif distance >= 2600:
        return '2600m以上'
    else:
        return 'その他'  # 範囲外の距離は'その他'として分類

# 距離を新しいカテゴリに変換
df_daiwa['距離分類'] = df_daiwa['距離'].apply(classify_distance)

# 距離分類ごとにグループ化
grouped = df_daiwa.groupby('距離分類')

# 成績を表示
def display_results(group):
    # 確定着順を元に成績を計算 (1着、2着、3着、馬券外)
    results = group['確定着順'].value_counts().reindex([1, 2, 3], fill_value=0)
    total = len(group)
    
    # 馬券外の数 (1, 2, 3以外)
    results[0] = total - results.sum()
    
    # 各割合を計算
    win_rate = results[1] / total * 100
    place_rate = (results[1] + results[2]) / total * 100
    show_rate = (results[1] + results[2] + results[3]) / total * 100
    
    # 出力形式: (1着-2着-3着-馬券外) + 勝率・連対率・複勝率
    return f"{results[1]}-{results[2]}-{results[3]}-{results[0]} | 勝率: {win_rate:.2f}% | 連対率: {place_rate:.2f}% | 複勝率: {show_rate:.2f}%"

# 距離分類別の成績を表示
for distance_category, group in grouped:
    print(f"距離分類: {distance_category} -> 成績: {display_results(group)}")


距離分類: 1000-1399m -> 成績: 196-206-218-1756 | 勝率: 8.25% | 連対率: 16.92% | 複勝率: 26.09%
距離分類: 1400-1799m -> 成績: 284-281-290-2334 | 勝率: 8.91% | 連対率: 17.72% | 複勝率: 26.81%
距離分類: 1800-2199m -> 成績: 113-153-116-1150 | 勝率: 7.38% | 連対率: 17.36% | 複勝率: 24.93%
距離分類: 2200-2600m -> 成績: 17-10-20-186 | 勝率: 7.30% | 連対率: 11.59% | 複勝率: 20.17%
距離分類: 2601m- -> 成績: 29-29-31-229 | 勝率: 9.12% | 連対率: 18.24% | 複勝率: 27.99%


/var/folders/tn/j8hdvbw52z938h6gxwsg4b4w0000gn/T/ipykernel_72475/1606024408.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_daiwa['距離分類'] = df_daiwa['距離'].apply(classify_distance)
